# Import Needed Libraries

In [ ]:
from statistics import mean,stdev
import ndlib.models.ModelConfig as mc
from termcolor import colored
import ndlib.models.epidemics as ep
import time
import collections
import heapq
import operator
import networkx as nx
import os
import os.path
import random
import numpy as np
from os import walk
from os import path
import sys
from utils import *
from variables import *

# Functions

In [ ]:
def get_top_k_best_nodes(dict_centrality, k):
    first_nodes = heapq.nlargest(k, dict_centrality, key=dict_centrality.get)    
    return first_nodes

In [ ]:
def get_bottom_k_best_nodes(dict_centrality, k):
    bottom_nodes = heapq.nsmallest(k, dict_centrality, key=dict_centrality.get)   
    return bottom_nodes

In [ ]:
def nth_moment(g,n):
    degree_np = np.array(list(dict(g.degree).values()))
    return (sum(degree_np**n)/len(g))

# SI Simulations for all Networks

In [15]:
#networks = next(walk(cleanNetworks), (None, None, []))[2]
cleanNetworks = '/SyntheticNetworks'
networks = ['networkovermu035.gml',
 'networkovermu030.gml',
 'networkovermu015.gml',
 'networkovermu010.gml',
 'networkovermu040.gml',
 'networkovermu005.gml',
 'networkovermu070.gml',
 'networkovermu025.gml',
 'networkovermu020.gml']

for network in networks:
    g = nx.read_gml(f'{cleanNetworks}/{network}',destringizer=int)
    n = network.split(".")[0]
    
    #Remove Self Loops
    g.remove_edges_from(nx.selfloop_edges(g))
    print(colored(f'Simulation currently on network {network} : {nx.info(g)}...', 'red',attrs=['bold']))
    #Take Largest Component
    nb_components = nx.number_connected_components(g)
    if(nb_components>1):
        subgraphs = list(nx.connected_components(g))
        max = subgraphs[0]
        for subgraph in subgraphs:
            if len(subgraph)>len(max):
                max = subgraph
        g = g.subgraphs(max)
    #print(network)
    
    #Initialize a dictionary for each centrality
    list_dicts = []

    for centrality in colors.keys():
        globals()[f'dict_{centrality}'] = dict()
        list_dicts.append(globals()[f'dict_{centrality}'])
        
    #get centrality results
    for dictionary,centrality in zip(list_dicts,colors.keys()):
        p = f'/data/SN/centrality_results/{n}/dict_{centrality}.txt'
        with open(p) as raw_data:
            for item in raw_data:
                if ':' in item:
                    key,value = item.split(':', 1)
                    value = value.replace('\n', '')
                    dictionary[int(key)]=float(value)
                else:
                    pass # deal with bad lines of text here
    
    #Initialize Average and Standard Deviation for Each Centrality
    for centrality in colors.keys():
        globals()[f'{centrality}_averageTime_each_simulation'] = dict()
        globals()[f'{centrality}_standardTime_each_simulation'] = dict()
        globals()[f'{centrality}_averageIterations_each_simulation'] = dict()
        globals()[f'{centrality}_standardIterations_each_simulation'] = dict()
        for fraction in fractions: 
            globals()[f'{centrality}_averageTime_each_simulation'][fraction] = 0
            globals()[f'{centrality}_standardTime_each_simulation'][fraction] = 0
            globals()[f'{centrality}_averageIterations_each_simulation'][fraction] = 0
            globals()[f'{centrality}_standardIterations_each_simulation'][fraction] = 0
    
    p = f'./Pickle Variables/{n}/'
    if(path.exists(p)==False):
        os.mkdir(p)
    network_size = len(g.nodes())
    save_obj(network_size,p,'networkSize')
    
    #Epidemic threshold
    firstMoment = nth_moment(g,1)
    secondMoment = nth_moment(g,2)
    epidemicThreshold = round(firstMoment/(secondMoment-firstMoment),3)
    save_obj(epidemicThreshold,p,'epidemicThreshold')

    
    for centrality in colors.keys(): 
        print(colored(f'Centrality: {centrality}', 'green',attrs=['bold']))
        for fraction in fractions:
            globals()[f'time_{centrality}'] = list()
            globals()[f'iterations_{centrality}'] = list()
            fraction_infected = round(fraction*network_size)
            if(centrality=="modv_neg"):
                infected_nodes = get_bottom_k_best_nodes(globals()[f'dict_{centrality}'], fraction_infected)
            else:
                infected_nodes = get_top_k_best_nodes(globals()[f'dict_{centrality}'], fraction_infected)
                
            
            
            for i in range(repeatedSimulations): 
                start = time.time()
                model = ep.SIModel(g)
                config = mc.Configuration()
                config.add_model_parameter('beta', epidemicThreshold)      
                config.add_model_initial_configuration("Infected", infected_nodes)
                model.set_initial_status(config)
                #print(infected_nodes)
                x= 0
                iterations = model.iteration_bunch(1)

                while((iterations[x]['node_count'][1])!=network_size):
                    x = x + 1
                    iterations = iterations + model.iteration_bunch(1)
                end = time.time()
                nbIterations = x
                takenTime = end - start
                globals()[f'time_{centrality}'].append(takenTime)
                globals()[f'iterations_{centrality}'].append(nbIterations)

            globals()[f'{centrality}_averageTime_each_simulation'][fraction] = mean(globals()[f'time_{centrality}'])
            globals()[f'{centrality}_standardTime_each_simulation'][fraction] = stdev(globals()[f'time_{centrality}'])
            globals()[f'{centrality}_averageIterations_each_simulation'][fraction] = mean(globals()[f'iterations_{centrality}'])
            globals()[f'{centrality}_standardIterations_each_simulation'][fraction] = stdev(globals()[f'iterations_{centrality}'])
            print(fraction,":",mean(globals()[f'time_{centrality}']))
            print(fraction,":",mean(globals()[f'iterations_{centrality}']))
        p = f"./Pickle Variables/{n}/beta-{epidemicThreshold}/"
        if(path.exists(p)==False):
            os.mkdir(p)
        save_obj(globals()[f'{centrality}_averageTime_each_simulation'], p,f'{centrality}_averageTime_each_simulation')
        save_obj(globals()[f'{centrality}_standardTime_each_simulation'], p, f'{centrality}_standardTime_each_simulation') 
        save_obj(globals()[f'{centrality}_averageIterations_each_simulation'], p,f'{centrality}_averageIterations_each_simulation')
        save_obj(globals()[f'{centrality}_standardIterations_each_simulation'], p, f'{centrality}_standardIterations_each_simulation') 

Simulation currently on network networkovermu035.gml : Name: 
Type: Graph
Number of nodes: 2500
Number of edges: 10417
Average degree:   8.3336...
Centrality: modv_pos
0.01 : 0.25890650272369387
0.01 : 25.07
0.02 : 0.2203380846977234
0.02 : 23.6
0.03 : 0.2091430926322937
0.03 : 23.25
0.04 : 0.19892622470855714
0.04 : 22.59
0.05 : 0.18895011901855469
0.05 : 21.59
0.06 : 0.19102285385131837
0.06 : 21.57
0.07 : 0.1869783329963684
0.07 : 20.66
0.08 : 0.18146368980407715
0.08 : 20.29
0.09 : 0.17769767999649047
0.09 : 20.1
0.1 : 0.1782592535018921
0.1 : 20.11
0.11 : 0.1678710722923279
0.11 : 19.45
0.12 : 0.16544594526290893
0.12 : 19.1
0.13 : 0.16694329738616942
0.13 : 19.09
0.14 : 0.16180622577667236
0.14 : 19.04
0.15 : 0.1597244167327881
0.15 : 18.97
0.16 : 0.15542999029159546
0.16 : 18.54
0.17 : 0.1518690323829651
0.17 : 18.26
0.18 : 0.14735944747924803
0.18 : 17.94
0.19 : 0.1492021584510803
0.19 : 18.47
0.2 : 0.14788628101348877
0.2 : 18.23
0.21 : 0.14434855699539184
0.21 : 17.9
0.22 : 0

0.02 : 0.22409627199172974
0.02 : 23.85
0.03 : 0.21367955684661866
0.03 : 22.79
0.04 : 0.20204420566558837
0.04 : 22.23
0.05 : 0.19160792112350464
0.05 : 21.54
0.06 : 0.1858122968673706
0.06 : 21.03
0.07 : 0.18380645513534546
0.07 : 20.85
0.08 : 0.17852792978286744
0.08 : 20.54
0.09 : 0.17593466281890868
0.09 : 20.35
0.1 : 0.17310936689376832
0.1 : 20.25
0.11 : 0.17085164785385132
0.11 : 20.33
0.12 : 0.16844676017761231
0.12 : 20.08
0.13 : 0.16591628551483154
0.13 : 19.88
0.14 : 0.16308988094329835
0.14 : 19.37
0.15 : 0.1620939826965332
0.15 : 19.17
0.16 : 0.15953302145004272
0.16 : 19.33
0.17 : 0.15470853567123413
0.17 : 18.78
0.18 : 0.15434205770492554
0.18 : 18.86
0.19 : 0.1513182330131531
0.19 : 18.79
0.2 : 0.15062901020050048
0.2 : 18.91
0.21 : 0.14475038766860962
0.21 : 17.99
0.22 : 0.14522401094436646
0.22 : 18.6
0.23 : 0.14173884630203248
0.23 : 18.2
0.24 : 0.1412920117378235
0.24 : 18.2
0.25 : 0.14079686641693115
0.25 : 18.27
0.26 : 0.1401519513130188
0.26 : 18.07
0.27 : 0.137

0.07 : 0.17574084997177125
0.07 : 20.21
0.08 : 0.1750694465637207
0.08 : 20.35
0.09 : 0.17014933824539186
0.09 : 20.12
0.1 : 0.16758435249328613
0.1 : 19.92
0.11 : 0.1622550058364868
0.11 : 19.35
0.12 : 0.1598011541366577
0.12 : 19.12
0.13 : 0.1578806757926941
0.13 : 19.1
0.14 : 0.15354456424713134
0.14 : 18.65
0.15 : 0.15329104661941528
0.15 : 18.53
0.16 : 0.15044347286224366
0.16 : 18.46
0.17 : 0.14693907976150514
0.17 : 18.07
0.18 : 0.14711362838745118
0.18 : 18.22
0.19 : 0.1428605031967163
0.19 : 17.69
0.2 : 0.14172892808914184
0.2 : 17.86
0.21 : 0.1409229063987732
0.21 : 17.78
0.22 : 0.13609432220458983
0.22 : 17.01
0.23 : 0.1350245475769043
0.23 : 16.93
0.24 : 0.13850900650024414
0.24 : 17.69
0.25 : 0.13406673908233643
0.25 : 17.18
0.26 : 0.13028825759887697
0.26 : 16.57
0.27 : 0.13099310874938966
0.27 : 16.82
0.28 : 0.1303553581237793
0.28 : 16.87
0.29 : 0.12971266746520996
0.29 : 16.75
0.3 : 0.12841559648513795
0.3 : 16.54
0.31 : 0.12726648807525634
0.31 : 16.54
0.32 : 0.125641

0.08 : 0.1751655411720276
0.08 : 19.65
0.09 : 0.16948815822601318
0.09 : 19.13
0.1 : 0.16231115102767946
0.1 : 18.82
0.11 : 0.1565821051597595
0.11 : 18.63
0.12 : 0.15208807706832886
0.12 : 18.16
0.13 : 0.1465844440460205
0.13 : 17.55
0.14 : 0.14599477529525756
0.14 : 17.68
0.15 : 0.14323634624481202
0.15 : 17.44
0.16 : 0.14046888351440429
0.16 : 17.12
0.17 : 0.13849119186401368
0.17 : 17.01
0.18 : 0.13656330108642578
0.18 : 17.04
0.19 : 0.1337197732925415
0.19 : 16.78
0.2 : 0.13701554536819457
0.2 : 16.94
0.21 : 0.13030970335006714
0.21 : 16.37
0.22 : 0.1279169225692749
0.22 : 16.21
0.23 : 0.12869050741195678
0.23 : 16.43
0.24 : 0.12663719415664673
0.24 : 16.13
0.25 : 0.12498391389846802
0.25 : 15.99
0.26 : 0.12378764390945435
0.26 : 15.84
0.27 : 0.122235586643219
0.27 : 15.62
0.28 : 0.1206741452217102
0.28 : 15.54
0.29 : 0.12047480583190918
0.29 : 15.67
0.3 : 0.11692676067352295
0.3 : 15.07
0.31 : 0.11637900590896606
0.31 : 15.11
0.32 : 0.11851353883743286
0.32 : 15.49
0.33 : 0.11736

0.13 : 0.16346824169158936
0.13 : 19.79
0.14 : 0.16230866432189942
0.14 : 19.66
0.15 : 0.1610255742073059
0.15 : 19.65
0.16 : 0.16129153966903687
0.16 : 19.56
0.17 : 0.16098692417144775
0.17 : 19.61
0.18 : 0.15837364196777343
0.18 : 19.28
0.19 : 0.15988444328308105
0.19 : 19.62
0.2 : 0.15876776695251466
0.2 : 19.37
0.21 : 0.1568721079826355
0.21 : 19.31
0.22 : 0.15630056142807006
0.22 : 19.18
0.23 : 0.15410481214523317
0.23 : 19.05
0.24 : 0.1511291241645813
0.24 : 18.63
0.25 : 0.1484274983406067
0.25 : 18.53
0.26 : 0.14783071994781494
0.26 : 18.74
0.27 : 0.14618773460388185
0.27 : 18.56
0.28 : 0.1449604296684265
0.28 : 18.54
0.29 : 0.1416664934158325
0.29 : 18.08
0.3 : 0.1421436548233032
0.3 : 18.3
0.31 : 0.14167563676834105
0.31 : 18.19
0.32 : 0.13759690284729004
0.32 : 17.75
0.33 : 0.14071710348129274
0.33 : 18.37
0.34 : 0.13609708309173585
0.34 : 17.76
0.35 : 0.13618050813674926
0.35 : 17.84
0.36 : 0.1359006094932556
0.36 : 17.88
0.37 : 0.133461594581604
0.37 : 17.59
0.38 : 0.134555

...

0.31 : 0.12775568246841432
0.31 : 17.16
0.32 : 0.12470934391021729
0.32 : 16.74
0.33 : 0.12861689567565918
0.33 : 17.54
0.34 : 0.12285154104232789
0.34 : 16.63
0.35 : 0.1244576072692871
0.35 : 16.89
0.36 : 0.12052808284759521
0.36 : 16.44
0.37 : 0.12251668453216552
0.37 : 16.32
0.38 : 0.12112613201141358
0.38 : 16.74
0.39 : 0.1191327714920044
0.39 : 16.37
0.4 : 0.12032351970672607
0.4 : 16.65
0.41 : 0.11777152538299561
0.41 : 16.26
0.42 : 0.11609139204025269
0.42 : 16.03
0.43 : 0.11630274534225464
0.43 : 16.14
0.44 : 0.11757484912872314
0.44 : 16.54
0.45 : 0.11984233140945434
0.45 : 16.95
0.46 : 0.11332539796829223
0.46 : 15.9
0.47 : 0.11145594358444214
0.47 : 15.74
0.48 : 0.11044830083847046
0.48 : 15.55
0.49 : 0.11209272146224976
0.49 : 15.92
0.5 : 0.11219217538833619
0.5 : 15.97
Centrality: kshell
0.01 : 0.2297883176803589
0.01 : 24.85
0.02 : 0.22251684665679933
0.02 : 24.41
0.03 : 0.20784555673599242
0.03 : 23.3
0.04 : 0.2017869234085083
0.04 : 23.28
0.05 : 0.2011828279495239
0.05 

0.36 : 0.12539440870285035
0.36 : 17.16
0.37 : 0.12168369770050048
0.37 : 16.65
0.38 : 0.1219343376159668
0.38 : 16.8
0.39 : 0.11832919597625732
0.39 : 16.25
0.4 : 0.11851629734039307
0.4 : 16.34
0.41 : 0.11691910266876221
0.41 : 16.16
0.42 : 0.1165695858001709
0.42 : 16.18
0.43 : 0.11604305267333985
0.43 : 16.22
0.44 : 0.11514568090438843
0.44 : 16.07
0.45 : 0.11516621589660644
0.45 : 16.15
0.46 : 0.11118798017501831
0.46 : 15.62
0.47 : 0.11083688735961914
0.47 : 15.64
0.48 : 0.10870678901672363
0.48 : 15.29
0.49 : 0.10611031770706177
0.49 : 15.03
0.5 : 0.10485515594482422
0.5 : 14.89
Centrality: degree
0.01 : 0.21359039783477785
0.01 : 23.77
0.02 : 0.20138420581817626
0.02 : 23.14
0.03 : 0.19488149404525756
0.03 : 22.77
0.04 : 0.1864779782295227
0.04 : 21.98
0.05 : 0.18336667776107787
0.05 : 21.85
0.06 : 0.1805860233306885
0.06 : 21.69
0.07 : 0.1789021110534668
0.07 : 21.74
0.08 : 0.1794571113586426
0.08 : 22.02
0.09 : 0.17523499488830566
0.09 : 21.54
0.1 : 0.17092737436294556
0.1 : 

0.37 : 0.11296741962432862
0.37 : 15.51
0.38 : 0.11260695934295654
0.38 : 15.53
0.39 : 0.11129522323608398
0.39 : 15.42
0.4 : 0.1079296851158142
0.4 : 14.93
0.41 : 0.10678761005401612
0.41 : 14.83
0.42 : 0.1097058129310608
0.42 : 15.31
0.43 : 0.10605309724807739
0.43 : 14.79
0.44 : 0.1062392783164978
0.44 : 14.95
0.45 : 0.10637970685958863
0.45 : 14.94
0.46 : 0.10471278190612793
0.46 : 14.75
0.47 : 0.10455750703811645
0.47 : 14.57
0.48 : 0.10394534826278687
0.48 : 14.73
0.49 : 0.10134748935699463
0.49 : 14.33
0.5 : 0.10186219215393066
0.5 : 14.45
Centrality: cbm
0.01 : 0.22464475631713868
0.01 : 25.12
0.02 : 0.20868004322052003
0.02 : 23.88
0.03 : 0.19834086656570435
0.03 : 22.95
0.04 : 0.19395516872406005
0.04 : 22.79
0.05 : 0.1904264807701111
0.05 : 22.66
0.06 : 0.18775779724121094
0.06 : 22.53
0.07 : 0.18475396394729615
0.07 : 22.35
0.08 : 0.17942360639572144
0.08 : 21.65
0.09 : 0.17827529430389405
0.09 : 21.68
0.1 : 0.1753088974952698
0.1 : 21.44
0.11 : 0.17644515752792359
0.11 : 2

0.42 : 0.14271148443222045
0.42 : 19.43
0.43 : 0.1403543496131897
0.43 : 19.16
0.44 : 0.13916764974594117
0.44 : 19.09
0.45 : 0.13852536916732788
0.45 : 19.1
0.46 : 0.13458867788314818
0.46 : 18.59
0.47 : 0.12965332508087157
0.47 : 17.85
0.48 : 0.1319019937515259
0.48 : 18.3
0.49 : 0.13058785438537598
0.49 : 18.26
0.5 : 0.1314310050010681
0.5 : 18.51
Centrality: cbc
0.01 : 0.2388429355621338
0.01 : 25.84
0.02 : 0.22216497421264647
0.02 : 24.67
0.03 : 0.21580966234207152
0.03 : 24.49
0.04 : 0.20937617778778075
0.04 : 23.97
0.05 : 0.206712646484375
0.05 : 23.58
0.06 : 0.20090727567672728
0.06 : 23.35
0.07 : 0.19754583597183228
0.07 : 23.07
0.08 : 0.19672818422317506
0.08 : 23.21
0.09 : 0.1900414538383484
0.09 : 22.44
0.1 : 0.18951545476913453
0.1 : 22.46
0.11 : 0.18824095726013185
0.11 : 22.41
0.12 : 0.18282498598098754
0.12 : 21.9
0.13 : 0.1810672950744629
0.13 : 21.81
0.14 : 0.18283427238464356
0.14 : 22.1
0.15 : 0.18050594568252565
0.15 : 21.97
0.16 : 0.1790170431137085
0.16 : 21.94
0

0.48 : 0.12367312669754028
0.48 : 17.65
0.49 : 0.11804696321487426
0.49 : 16.7
0.5 : 0.12000234127044677
0.5 : 17.2
Centrality: map_eq
0.01 : 0.2233784818649292
0.01 : 24.84
0.02 : 0.20545963764190675
0.02 : 23.44
0.03 : 0.20259264945983887
0.03 : 23.31
0.04 : 0.19626923084259032
0.04 : 23.14
0.05 : 0.19080156803131104
0.05 : 22.55
0.06 : 0.1857130217552185
0.06 : 21.98
0.07 : 0.18446091175079346
0.07 : 22.11
0.08 : 0.18358131885528564
0.08 : 21.93
0.09 : 0.1803981375694275
0.09 : 21.93
0.1 : 0.1785864996910095
0.1 : 21.89
0.11 : 0.17564522981643677
0.11 : 21.64
0.12 : 0.1724017333984375
0.12 : 21.22
0.13 : 0.17368320465087891
0.13 : 21.35
0.14 : 0.17084524393081665
0.14 : 21.26
0.15 : 0.1705924940109253
0.15 : 21.36
0.16 : 0.16874029159545897
0.16 : 21.19
0.17 : 0.16535043478012085
0.17 : 20.76
0.18 : 0.16710959434509276
0.18 : 21.11
0.19 : 0.16781793355941774
0.19 : 21.16
0.2 : 0.1585874390602112
0.2 : 20.02
0.21 : 0.16502754211425782
0.21 : 20.1
0.22 : 0.16464212656021118
0.22 : 20.